# Access to Prompt data

- creationn date : 2025-12-04

In [ ]:
from lsst.daf.butler import Butler
from lsst.daf.butler import CollectionType
from lsst.daf.butler import DatasetType
import textwrap

try:
    from lsst.daf.butler.exceptions import DatasetTypeError
    print("Import de 'DatasetTypeError' réussi depuis lsst.daf.butler.exceptions.")
except ImportError:
    # Si le sous-module 'exceptions' n'existe pas ou l'exception a été renommée/déplacée,
    # nous allons devoir capturer l'exception parent la plus générale levée par le Butler.
    print("Avertissement : 'DatasetTypeError' non trouvé dans lsst.daf.butler.exceptions. Utilisation d'une gestion d'erreurs plus générale.")
    
    # Définir l'exception de rechange (un alias vers un type d'erreur plus générique du Butler)
    # L'exception qui est levée est souvent une sous-classe de RuntimeError ou d'une autre erreur du Butler.
    # Pour le test de getDatasetType, nous allons tenter de capturer l'exception directement.
    pass # Nous n'avons pas besoin de définir un alias si nous gérons l'exception après la tentative.

# --- Le reste de votre code (en utilisant l'exception directement après la tentative) ---


# Ouvrir le repo
#repo = "/repo/main"
REPO_URI = "/repo/main"
butler = Butler(REPO_URI)

# Lister toutes les collections et filtrer celles contenant "LSSTCam/prompt"
collections = butler.registry.queryCollections()

# find collections which has prompt in it
prompt_colls = [c for c in collections if "LSSTCam/prompt" in c]


# 1. Pré-filtrage : Crée une nouvelle liste sans les collections contenant 'config'
filtered_colls = [c for c in prompt_colls if 'config' not in c]

# 2. Itération sur la liste filtrée
for c in filtered_colls:
    ctype = butler.registry.getCollectionType(c)
    print(f"{c:90s} {CollectionType(ctype).name}")

In [ ]:
# Le REPO_URI est le chemin du dépôt (comme dans votre commande bash)
#REPO_URI = "/repo/main"

# La collection de données spécifique à cibler
#COLLECTION_NAME = 'LSSTCam/prompt/output-2025-11-04/SingleFrame/pipelines-682fa38-config-8f017ea'
COLLECTION_NAME = "LSSTCam/prompt/output-2025-11-20"

# Initialisation du Butler
butler = Butler(REPO_URI, collections=[COLLECTION_NAME])

print(f"Butler initialisé pour la collection : {COLLECTION_NAME}")

In [ ]:
# Assurez-vous d'avoir bien initialisé le Butler (Butler est déjà initialisé dans le contexte)
registry = butler.registry
dataset_types = list(registry.queryDatasetTypes())

COLLECTION_NAME = butler.collections[0] # Récupère la collection par défaut
#COLLECTION_NAME = butler.collection.default

print(f"La collection '{COLLECTION_NAME}' contient {len(dataset_types)} types de datasets.")

# Afficher les noms des types de datasets, en filtrant ceux liés à la détection (DIA)
print("\nTypes de Datasets pertinents (contenant 'DIA' ou 'Source') :")


# --- NOUVEAU FILTRE : Types de données pertinents ---
# Classes de stockage que nous voulons conserver (tables de données)
#ALLOWED_STORAGE_CLASSES = ['DataFrame', 'SourceCatalog', 'Catalog']
ALLOWED_STORAGE_CLASSES = ['DataFrame']

# Mots-clés pour filtrer les datasets liés à la détection et à l'alerte
ALLOWED_KEYWORDS = ['DIA', 'Source', 'Dia', 'DiaObject','dia'] 

print(f"\nTypes de Datasets pertinents (filtrés par Nom et Classe de Stockage {ALLOWED_STORAGE_CLASSES}):")


keep_dataset_types = []
keep_dataset_dimensions = []

# loop on datasettypes
for ds_type in dataset_types:
    
    ds_name = ds_type.name
    
    # Correction de l'erreur: Utilisation d'un bloc try/except pour gérer les 
    # "KeyError" qui se produisent lorsque le Butler ne peut pas résoudre la classe de stockage (ex: 'SpectractorWorkspace').
    storage_class_name = "N/A"
    try:
        if ds_type.storageClass:
            storage_class_name = ds_type.storageClass.name
    except KeyError:
        # La classe existe dans le registre mais le module Python n'est pas chargé
        storage_class_name = "UNRESOLVED_CLASS" 
    except Exception as e:
        # Pour tout autre type d'erreur
        print(f"Avertissement: Échec de la résolution de la classe de stockage pour {ds_name}. Erreur: {e}")
        
    
    # Filtre 1: Doit contenir un mot-clé pertinent dans le nom
    # Utilisation de .upper() pour une comparaison insensible à la casse dans le nom du dataset
    is_relevant_keyword = any(keyword.upper() in ds_name.upper() for keyword in ALLOWED_KEYWORDS)
    
    # Filtre 2: Doit être une classe de stockage de type catalogue/table
    is_relevant_storage = storage_class_name in ALLOWED_STORAGE_CLASSES
    
    if is_relevant_keyword and is_relevant_storage:
        # Utilisation de textwrap pour gérer les noms de datasets longs sans tronquer la sortie
        wrapped_name = textwrap.shorten(ds_name, width=40, placeholder='...')
        print(f"  - **{wrapped_name:40s}** : Stocké comme un {storage_class_name}")
        keep_dataset_types.append(ds_name)
        required_dimensions = list(ds_type.dimensions.names)
        keep_dataset_dimensions.append(required_dimensions)

print("\n--- Liste des catalogues à utiliser pour l'extraction de données : ---")
print(keep_dataset_types)

In [ ]:
print(f"Butler initialisé. Les collections par défaut sont : {butler.collections}")
print(f"Le Butler utilise la collection par défaut : {butler.collections[0]}")

for dataset_type_name in keep_dataset_types:

    print(f"Test de l'existence du DatasetType '{dataset_type_name}'...")

    # ----------------------------------------------------
    # 💡 ÉTAPE 1 : VÉRIFIER L'EXISTENCE DU DATASET TYPE
    # ----------------------------------------------------
    try:
        # Tente de récupérer l'objet DatasetType. 
        dt = butler.registry.getDatasetType(dataset_type_name)
        required_dimensions = dt.dimensions.names
        print(f"✅ Le type '{dt}' est présent et stocké comme : {dt.storageClass.name}, required dimensions : {required_dimensions}")
     
    
        # ... Les étapes 2 et 3 (queryDataIds et butler.get) suivent ici ...


        # ----------------------------------------------------
        # 💡 ÉTAPE 2 : S'IL EXISTE, CHERCHER LES DATAIDS
        # ----------------------------------------------------
        print("Tentative de recherche d'identifiants de données (DataIds)...")

        # Recherche des DataIds existants pour ce type de dataset
        #valid_diasource_ids = list(butler.registry.queryDataIds(datasets=diasource_type, dimensions=['visit', 'detector', 'band']))
        #valid_diasource_ids = list(butler.registry.queryDataIds(datasets=diasource_type),dimensions=required_dimensions,limit=1)  # Suppression du paramètre 'dimensions'
        #valid_diasource_ids = list(butler.registry.queryDataIds(datasets=diasource_type),dimensions=required_dimensions,limit=1)

        valid_data_ids = list(butler.registry.queryDataIds(
        datasets=dataset_type_name, 
        dimensions=required_dimensions, # <- Correctement passé
        ).limit(1))

        if valid_data_ids:
            print(f"✅ {len(valid_data_ids)} combinaisons visit/detector/band trouvées.")

            #required_dimensions = list(ds_type_obj.dimensions.getDimensionNames())
            
            # Choisir le premier dataId trouvé
            #target_data_id = valid_diasource_ids[0].butlerDataId
            target_data_id = valid_data_ids[0].getButlerDataId()
            print(f"DataId cible pour le chargement : {target_data_id}")

            # ----------------------------------------------------
            # 💡 ÉTAPE 3 : TENTER LE CHARGEMENT
            # ----------------------------------------------------
            try:
                diasource_catalogue = butler.get(dataset_type_name, dataId=target_data_id)
                print("\n--- ✅✅ Chargement dataset  Réussi ---")
                print(f"✅✅ Catalogue chargé avec **{len(diasource_catalogue)}** entrées.")
            
            except Exception as e:
                # Gère les erreurs qui pourraient survenir lors de la lecture physique (I/O, corruption, etc.)
                print(f"❌ Erreur I/O ou de lecture physique (butler.get()) : {e}")
            
        else:
            print(f"❌ Attention : Le type '{dataset_type_name}' existe, mais aucune donnée n'a été produite (0 DataId trouvée).")

        

    # Si l'importation de DatasetTypeError a réussi, vous pouvez l'utiliser :
    # except DatasetTypeError: 
    #     ...

    # Alternativement (et plus robuste sans importation réussie), 
    # capturer l'exception spécifique levée par la méthode getDatasetType
    # ou une erreur plus générale si la première échoue.

    except Exception as e:
        # Si getDatasetType lève une exception (comme une RuntimeError ou l'exception non importée)
        # qui contient le message que le dataset type n'existe pas, nous la gérons ici.
    
        # La méthode la plus sûre est de vérifier le message de l'erreur levée par le Butler, 
        # car l'erreur est levée au moment où getDatasetType échoue.
        if f"Dataset type '{dataset_type_name}' does not exist" in str(e):
             print(f"❌ Le type de dataset '{diasource_type}' n'existe pas dans la collection par défaut (Erreur capturée : {type(e).__name__}).")
             print("Veuillez choisir une collection issue d'un pipeline d'alertes (ApPipe complet).")
        else:
             # Gérer une autre erreur inattendue
             raise e
# ----------------------------------------------------